In [1]:
import os
%load_ext autoreload
%autoreload 2
os.chdir("..")
os.chdir("..")
os.chdir("..")

In [55]:
import os
import tkinter as tk
from tkinter import messagebox
from pathlib import Path
import pandas as pd
from PIL import Image, ImageTk
import json
import pygame
import random
from datetime import datetime
from deep_translator import GoogleTranslator
from random import randint


from app.utils.data_loader import DataLoader
from app.utils.game_timer import GameTimer
from app.utils.save_data import save_game_data

# Inicializa o mixer do pygame
pygame.mixer.init()

TITLE_FONT = ("Arial", 20)
LABEL_FONT = ("Arial", 14)
SMALL_FONT = ("Arial", 12)

# DATA_PATH = Path("extract_data_video/data/extracted_data/words/data_organize")
DATA_PATH = Path("database/vocabulary/words/data_organize")
SUBCATEGORY_NAME = "em_loja"


class HangmanGame(tk.Frame):
    MAX_ATTEMPTS = 6

    def __init__(self, parent=None, **kwargs):
        super().__init__(parent, **kwargs)
        self.parent = parent
        self.loader = DataLoader(base_path=DATA_PATH)
        self.list_data_words:list[dict] = self.loader.get_all_words(subcategory_name=SUBCATEGORY_NAME)
        random.shuffle(self.list_data_words)

        self.timer = GameTimer(self)

        self.clicks_on_guess = 0

        self.last_word = None

        self.id_game = self.gerar_hash_id()
        self.reset_game()
        self.setup_ui()

    def gerar_hash_id(self):
        agora = datetime.now().strftime("%Y%m%d%H%M%S%f")  # AnoMesDiaHoraMinSegMicroseg
        return hex(abs(hash(agora)))[2:]  # Converte para hexadecimal e remove '0x'

    def reset_game(self):
        if not self.list_data_words:
            messagebox.showinfo("Fim", "Não há mais palavras disponíveis.")
            self.parent.quit()
            return

        self.dict_info_words = self.list_data_words.pop(random.randint(0, len(self.list_data_words)-1))

        self.word_answer = self.dict_info_words.get("text_eng", None)
        
        self.word_question = self.dict_info_words.get("text_pt_br", None) 

        self.guessed_word = self.hide_text(self.word_answer) 
        self.remaining_attempts = self.MAX_ATTEMPTS
        self.guessed_letters = set()

        self.timer.play()

    def hide_text(self, text: str) -> str:
        # return [" " if c == " " else "_" for c in text]
        return ['_' if c.isalpha() else c for c in text]

    def show_label_words(self):
        font=("Arial", 12)
        fg="black"
        self.palavras_restantes_label = tk.Label(self, text=f"Palavras restantes: {len(self.list_data_words)}", font=font, fg=fg)
        self.palavras_restantes_label.place(relx=0, rely=0.0, anchor="nw")

    def setup_ui(self):

        self.show_label_words()

        self.word_label = tk.Label(self, text=" ".join(self.guessed_word), font=TITLE_FONT)
        self.word_label.pack(pady=10)

        self.attempts_label = tk.Label(self, text="", font=LABEL_FONT)
        self.attempts_label.pack()

        self.guessed_label = tk.Label(self, text="", font=SMALL_FONT)
        self.guessed_label.pack()

        self.image_label = tk.Label(self)
        self.image_label.pack(pady=10)

        self.hint_label = tk.Label(self, text="", font=LABEL_FONT)
        self.hint_label.pack()

        self.input_frame = tk.Frame(self)
        self.input_frame.pack(pady=10)

        tk.Label(self.input_frame, text="Letra:", font=SMALL_FONT).pack(side="left")
        self.letter_entry = tk.Entry(self.input_frame, width=5, font=LABEL_FONT)
        self.letter_entry.pack(side="left")

        self.guess_button = tk.Button(self.input_frame, text="Adivinhar", command=self.check_guess)
        self.guess_button.pack(side="left", padx=5)

        self.update_ui()

    def update_ui(self):
        
        self.palavras_restantes_label.config(text=f"Palavras restantes: {len(self.list_data_words)}")

        self.word_label.config(text=" ".join(self.guessed_word))
        self.attempts_label.config(text=f"Tentativas restantes: {self.remaining_attempts}")
        self.guessed_label.config(text=f"Letras tentadas: {', '.join(sorted(self.guessed_letters))}")
        self.hint_label.config(text=f"Dica: {self.word_question}")

        # if not img_path.exists():
        #     print(f"[ERRO] Caminho da imagem não encontrado: {img_path}")

        # if not hasattr(self.image_label, 'current_image_path') or self.image_label.current_image_path != img_path:

        image_figure = self.dict_info_words.get("image_figure", None)

        if image_figure:
            img_path = image_figure
            img = Image.open(img_path).resize((200, 200))
            photo = ImageTk.PhotoImage(img)

            self.image_label.config(image=photo)
            self.image_label.image = photo  # <- MANTÉM REFERÊNCIA
        else:
            self.image_label.config(image='')
            self.image_label.image = None

    def check_guess(self):
        self.clicks_on_guess += 1

        guess = self.letter_entry.get().strip().lower()
        self.letter_entry.delete(0, tk.END)

        if not guess.isalpha():
            messagebox.showwarning("Letra inválida", "Digite apenas letras.")
            return

        for letter in guess:
            if letter in self.guessed_letters:
                continue

            self.guessed_letters.add(letter)

            if letter in self.word_answer:
                self.reveal_letters(letter)
            else:
                self.remaining_attempts -= 1

            if self.check_game_end():
                return

        self.update_ui()

    def reveal_letters(self, letter):
        for idx, char in enumerate(self.word_answer):
            if char == letter:
                self.guessed_word[idx] = letter

    def check_game_end(self) -> bool:
        if "_" not in self.guessed_word:
            self.save_score(True)
            messagebox.showinfo("Você venceu!", f"A palavra era: {self.word_answer}")
            self.restart_game()
            return True

        if self.remaining_attempts <= 0:
            self.save_score(False)

            # Coloca a palavra de volta na lista
            self.list_data_words.append(self.dict_info_words)
            
            messagebox.showerror("Fim de jogo", f"A palavra era: {self.word_answer}")
            self.restart_game()
            return True

        return False

    def restart_game(self):
        self.last_word = self.dict_info_words.copy()
        self.reset_game()
        self.update_ui()

    def save_score(self, won: bool):
        attempts_used = self.MAX_ATTEMPTS - self.remaining_attempts
        difficulty = attempts_used / self.MAX_ATTEMPTS

        correct_guessed_letters=[letter for letter in self.guessed_word if letter.isalnum()]
        incorrect_guessed_letters = list(set(self.guessed_letters).difference(set(self.guessed_word)))

        # finaliza_tempo
        # end_time = time.time()
        time_taken = self.timer.elapsed_time

        clicks_on_guess = self.clicks_on_guess

        path_word = self.dict_info_words.get("path", None)
        category = path_word.parts[-3]
        sub_category = path_word.parts[-2]
        # adjetivos/sobre_as_pessoas/unable

        if not any([category == loader_category.name for loader_category in self.loader.get_categories()]):
            print("Categoria Não Existe")

        self.timer.reset_timer()
        self.clicks_on_guess = 0

        # assert total_attempts >= used_attempts, "Tentativas usadas não podem ser maiores que o total permitido."

        datetime_now = datetime.now().isoformat(timespec="seconds")
        

        game_data = {
            "id_game": self.id_game,
            "datetime": datetime_now,
            "word": self.word_answer,
            "category":category,
            "sub_category":sub_category,
            "hint": self.word_question,
            "won": won,
            "difficulty": difficulty,
            "total_attempts": self.MAX_ATTEMPTS,
            "used_attempts": attempts_used,
            "clicks_on_guess": clicks_on_guess,
            "correct_guessed_letters": correct_guessed_letters,
            "incorrect_guessed_letters": incorrect_guessed_letters,
            # "correct_guesses": correct_guesses,
            # "incorrect_guesses": incorrect_guesses,
            "time_taken": time_taken,
            "game_name": "hangman",
        }

        save_game_data(game_data)
        # self.save_score()

    def translate_sentence(self, sentence, source_lang="en", target_lang="pt"):
        try:
            translator = GoogleTranslator(source=source_lang, target=target_lang)
            return translator.translate(sentence)
        except Exception as e:
            print(f"Translation error: {e}")
            return "Translation unavailable"

if __name__ == "__main__":
    try:
        root = tk.Tk()
        root.title("Jogo da Forca com Imagens")
        game = HangmanGame(root)
        game.pack(expand=True, fill="both")
        root.geometry("600x600")
        root.mainloop()
    except Exception as e:
        print(f"Error {e}")
    
    finally:
        root.destroy()


TclError: can't invoke "destroy" command: application has been destroyed

In [ ]:
from app.utils.data_loader import DataLoader
from pathlib import Path
import os

kind = "phrases"
DATA_PATH = "database/extract_data_video/data/extracted_data/{kind}/data_organize"
SUBCATEGORY_NAME = "em_loja"

os.listdir(DATA_PATH.format(kind=kind))
loader = DataLoader(base_path=DATA_PATH.format(kind=kind))
loader.get_categories(kind_return = "dict").keys()

dict_keys(['viagem', 'compras', 'restaurante_e_bar', 'encontros', 'avião', 'saúde', 'hotel', 'tempo_e_clima', 'comida', 'frases_básicas', 'emergências', 'serviços', 'educação', 'meios_de_comunicação', 'empregos', 'outro_transporte', 'conversa_geral', 'carro', 'fazer_amigos', 'tempo_de_lazer'])

In [15]:
estrutura = {}
estrutura["cate"] = "teste"


estrutura

{'cate': 'teste'}

In [29]:
estrutura = {}
list_kinds = ["words", "phrases"]
for kind in list_kinds:
    loader = DataLoader(base_path=DATA_PATH.format(kind=kind))
    estrutura[kind] = {}
    for categoria in loader.get_categories():
        estrutura[kind][categoria.name] = {}
        for subcat in loader.get_subcategories(categoria):
            estrutura[kind][categoria.name][subcat.name] = []
            for word_path in loader.get_word_paths(subcat):
                estrutura[kind][categoria.name][subcat.name].append(word_path.name)
            


In [31]:
estrutura["words"]

{'pessoas': {'identidade': ['status',
   'privacy',
   'passport',
   'age',
   'birth_certificate',
   'missus',
   'name',
   'habit',
   'citizenship',
   'nationality',
   'autograph',
   'pseudonym',
   'personality',
   'human',
   'reputation',
   'miss',
   'last_name',
   'motherland',
   'person',
   'fingerprint',
   'signature',
   'mister',
   'nickname'],
  'idade_e_eventos_da_vida': ['long-lived',
   'to_be_born',
   'young',
   'ceremony',
   'to_age',
   'birthday',
   'honeymoon',
   'death',
   'fireworks',
   'to_die',
   'gift',
   'baby',
   'to_celebrate',
   'wedding',
   'festival',
   'child',
   'middle-aged',
   'engagement',
   'old',
   'teenager',
   'occasion',
   'parade',
   'life',
   'to_congratulate',
   'to_marry',
   'invitation',
   'divorce',
   'anniversary',
   'tradition',
   'mature',
   'surprise',
   'to_retire',
   'adult'],
  'familia': ['hereditary',
   'to_adopt',
   'triplets',
   'sister',
   'brother',
   'daughter',
   'related',
 

In [ ]:
{"words": {
        "Casa": {
            "Partes": ["porta", "janela", "telhado"],
            "Cômodos": ["sala", "quarto", "cozinha"]
        },}

In [33]:
import tkinter as tk

class WordBaseApp(tk.Frame):
    def __init__(self, parent=None, **kwargs):
        super().__init__(parent, **kwargs)
        self.parent = parent

        self.data_path = "database/extract_data_video/data/extracted_data/{kind}/data_organize"
        self.current_type = "words"
        # self.data = {
        #     "words": {
        #         "Casa": {
        #             "Partes": ["porta", "janela", "telhado"],
        #             "Cômodos": ["sala", "quarto", "cozinha"]
        #         },
        #         "Trabalho": {
        #             "Profissões": ["engenheiro", "professor"]
        #         }
        #     },
        #     "phrases": {
        #         "Saudações": {
        #             "Informal": ["Oi!", "E aí?"],
        #             "Formal": ["Bom dia", "Como vai?"]
        #         }
        #     }
        # }

        self.data = self.create_estructure()

        self.build_ui()

    def create_estructure(self):
        estrutura = {}
        list_kinds = ["words", "phrases"]
        for kind in list_kinds:
            loader = DataLoader(base_path=DATA_PATH.format(kind=kind))
            estrutura[kind] = {}
            for categoria in loader.get_categories():
                estrutura[kind][categoria.name] = {}
                for subcat in loader.get_subcategories(categoria):
                    estrutura[kind][categoria.name][subcat.name] = []
                    for word_path in loader.get_word_paths(subcat):
                        estrutura[kind][categoria.name][subcat.name].append(word_path.name)
        return estrutura
    

    def build_ui(self):
        # Limpa tudo
        for widget in self.winfo_children():
            widget.destroy()

        # Topo: Botões de tipo
        top_frame = tk.Frame(self)
        top_frame.pack(pady=10)

        words_btn = tk.Button(top_frame, text="Palavras", command=lambda: self.switch_type("words"))
        words_btn.pack(side="left", padx=5)

        phrases_btn = tk.Button(top_frame, text="Frases", command=lambda: self.switch_type("phrases"))
        phrases_btn.pack(side="left", padx=5)

        # Área de conteúdo
        self.content_frame = tk.Frame(self)
        self.content_frame.pack(fill="both", expand=True, pady=10)

        self.draw_categories()

    def switch_type(self, type_name):
        self.current_type = type_name
        self.draw_categories()

    def draw_categories(self):
        for widget in self.content_frame.winfo_children():
            widget.destroy()

        categories = self.data.get(self.current_type, {})

        print(self.current_type)
        print("-----")
        print(categories.items())

        os.listdir(self.data_path.format(kind=self.current_type))
        loader = DataLoader(base_path=DATA_PATH.format(kind=kind))
        loader.get_categories(kind_return = "dict").keys()
        

        for category, subcats in categories.items():
            cat_frame = tk.Frame(self.content_frame)
            cat_frame.pack(fill="x", padx=10, pady=5)

            is_expanded = tk.BooleanVar(value=False)

            def toggle(sub_frame=subcats, var=is_expanded, master=cat_frame, cat_name=category):
                if var.get():
                    for widget in master.winfo_children()[1:]:
                        widget.destroy()
                    var.set(False)
                else:
                    for sub, items in sub_frame.items():
                        
                        sub_btn = tk.Button(
                            master,
                            text=f"   {sub}",
                            anchor="w",
                            command=lambda sub=sub, items=items: self.show_word_list(category=cat_name, subcategory=sub, words=items)
                        )
                        sub_btn.pack(fill="x")
                    var.set(True)

            btn = tk.Button(cat_frame, text=category, anchor="w", command=toggle)
            btn.pack(fill="x")

    def show_word_list(self, category, subcategory, words):
        # Limpa conteúdo

        print(f"sub:{category}, items:{subcategory}")
        
        for widget in self.winfo_children():
            widget.destroy()

        # Botão voltar
        back_btn = tk.Button(self, text="Voltar", command=self.build_ui)
        back_btn.pack(pady=10)


        # Título
        title = tk.Label(self, text=f"{self.current_type.title()} - {category} > {subcategory}", font=("Arial", 16))
        title.pack(pady=5)

        # Lista de palavras
        list_frame = tk.Frame(self)
        list_frame.pack(fill="both", expand=True, pady=10)

        for word in words:
            word_label = tk.Label(list_frame, text=word, anchor="w")
            word_label.pack(fill="x", padx=20, pady=2)



if __name__ == "__main__":
    root = tk.Tk()
    root.title("Base de Palavras e Frases")
    root.geometry("600x600")
    app = WordBaseApp(root)
    app.pack(fill="both", expand=True)  # <- Mostra o app na janela
    root.mainloop()



words
-----
dict_items([('pessoas', {'identidade': ['status', 'privacy', 'passport', 'age', 'birth_certificate', 'missus', 'name', 'habit', 'citizenship', 'nationality', 'autograph', 'pseudonym', 'personality', 'human', 'reputation', 'miss', 'last_name', 'motherland', 'person', 'fingerprint', 'signature', 'mister', 'nickname'], 'idade_e_eventos_da_vida': ['long-lived', 'to_be_born', 'young', 'ceremony', 'to_age', 'birthday', 'honeymoon', 'death', 'fireworks', 'to_die', 'gift', 'baby', 'to_celebrate', 'wedding', 'festival', 'child', 'middle-aged', 'engagement', 'old', 'teenager', 'occasion', 'parade', 'life', 'to_congratulate', 'to_marry', 'invitation', 'divorce', 'anniversary', 'tradition', 'mature', 'surprise', 'to_retire', 'adult'], 'familia': ['hereditary', 'to_adopt', 'triplets', 'sister', 'brother', 'daughter', 'related', 'nephew', 'mother', 'grandmother', 'grandson', 'son-in-law', 'wife', 'aunt', 'niece', 'brother-in-law', 'son', 'childless', 'adoptive_parents', 'father', 'patern

## Futuras features

- taxa de erros
- ultimas palavras [acerto/erro] [audio]
- palavras já vistas [gral de conhecimento]
- completude por categoria, depois subcategoria [conhecidas/desconhecidas]

- melhores formas de esconher as caregorias pra jogar 

- criar paineis de gráfico

In [1]:
import tkinter as tk

def ouvir_audio():
    print("Tocando áudio...")  # Aqui você chamaria a função real de áudio
    popup.destroy()

def mostrar_mensagem():
    global popup
    popup = tk.Toplevel(root)
    popup.title("Mensagem")
    popup.geometry("300x150")
    popup.grab_set()  # Faz a janela se comportar como modal

    label = tk.Label(popup, text="Você errou a palavra!")
    label.pack(pady=10)

    botao_ouvir = tk.Button(popup, text="🔊 Ouvir áudio", command=ouvir_audio)
    botao_ouvir.pack(pady=5)

    botao_ok = tk.Button(popup, text="OK", command=popup.destroy)
    botao_ok.pack(pady=5)

root = tk.Tk()
tk.Button(root, text="Mostrar mensagem", command=mostrar_mensagem).pack(pady=20)
root.mainloop()


In [2]:
import tkinter as tk
from tkinter import messagebox

# Funções dos botões
def marcar_errado():
    messagebox.showinfo("Ação", "Palavra marcada como errada.")

def editar_palavra():
    messagebox.showinfo("Ação", "Abrindo editor de atributos...")

def favoritar_palavra():
    messagebox.showinfo("Ação", "Palavra favoritada!")

# Criação da janela
root = tk.Tk()
root.title("Exemplo de Botões com Emojis")
root.geometry("300x150")

# Frame para os botões
frame = tk.Frame(root)
frame.pack(pady=20)

# Botões com emojis (pode ajustar os emojis se preferir)
btn_errado = tk.Button(frame, text="❌ | Errado", command=marcar_errado)
btn_editar = tk.Button(frame, text="✏️ Editar", command=editar_palavra)
btn_favoritar = tk.Button(frame, text="⭐ Favoritar", command=favoritar_palavra)

# Posicionamento
btn_errado.grid(row=0, column=0, padx=5)
btn_editar.grid(row=0, column=1, padx=5)
btn_favoritar.grid(row=0, column=2, padx=5)

# Loop principal
root.mainloop()


'01/05/2025, 07:19:08'

---

In [3]:
import tkinter as tk

class App(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Exemplo de Mudar de Tela")
        self.geometry("400x300")

        # Tela 1
        self.tela_inicial = tk.Frame(self)
        self.tela_inicial.pack(fill="both", expand=True)

        label = tk.Label(self.tela_inicial, text="Tela Inicial", font=("Arial", 18))
        label.pack(pady=20)

        botao_ir_para_tela2 = tk.Button(self.tela_inicial, text="Ir para Tela 2", command=self.mostrar_tela2)
        botao_ir_para_tela2.pack()

        # Tela 2
        self.tela2 = tk.Frame(self)

        label2 = tk.Label(self.tela2, text="Você está na Tela 2!", font=("Arial", 18))
        label2.pack(pady=20)

        botao_voltar = tk.Button(self.tela2, text="Voltar para Tela Inicial", command=self.mostrar_tela_inicial)
        botao_voltar.pack()

    def mostrar_tela2(self):
        self.tela_inicial.pack_forget()
        self.tela2.pack(fill="both", expand=True)

    def mostrar_tela_inicial(self):
        self.tela2.pack_forget()
        self.tela_inicial.pack(fill="both", expand=True)

if __name__ == "__main__":
    app = App()
    app.mainloop()


In [ ]:
import pandas as pd
import os

In [ ]:
os.chdir("..")

In [ ]:
df = pd.read_csv("database/csv/words_oxford_levels.csv")

WORDS = df.sample(100)["Word"].tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'database/csv/words_oxford_levels.csv'

In [ ]:
from deep_translator import GoogleTranslator



WORDS = ['Red', 'Blue', 'Green', 'Pink', 'Black',
                        'Yellow', 'Orange', 'White', 'Purple', 'Brown']